Applying LLM on Data

In [1]:
#Install the following packages required to execute this notebook.

%pip install --upgrade --user google-cloud-bigquery-connection google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 16.9 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
# Set the project and BigQuery region

PROJECT_ID = "de-gcp-ema-con-e7906-npd-1"  # @param {type:"string"}
REGION = "US"  # @param {type: "string"}

In [2]:
# Set up project variables

DATASET_ID = "bqml_llm"
CONN_NAME = "bqml_llm_conn"
LLM_MODEL_NAME = "bqml-vertex-llm"

In [8]:
#Common setup instructions - For both PaLM API and Gemini API in Vertex AI, the setup process is the same

%pip install --upgrade --user google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [10]:
import vertexai

vertexai.init(project="de-gcp-ema-con-e7906-npd-1", location="us-central1")

In [11]:
# Import Libraries

from google.cloud import bigquery
from google.cloud import bigquery_connection_v1 as bq_connection
import pandas as pd
pd.set_option('display.max_colwidth', 1000)

In [12]:
# Create BQ Cloud resource connection

client = bq_connection.ConnectionServiceClient()
new_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}"
exists_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
cloud_resource_properties = bq_connection.CloudResourceProperties({})

# Try to use an existing connection if one already exists. If not, create a new one.
try:
    request = client.get_connection(
        request=bq_connection.GetConnectionRequest(name=exists_conn_parent)
    )
    CONN_SERVICE_ACCOUNT = f"serviceAccount:{request.cloud_resource.service_account_id}"
except Exception:
    connection = bq_connection.types.Connection(
        {"friendly_name": CONN_NAME, "cloud_resource": cloud_resource_properties}
    )
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": new_conn_parent,
            "connection_id": CONN_NAME,
            "connection": connection,
        }
    )
    response = client.create_connection(request)
    CONN_SERVICE_ACCOUNT = (
        f"serviceAccount:{response.cloud_resource.service_account_id}"
    )
print(CONN_SERVICE_ACCOUNT)

serviceAccount:bqcx-179499000957-x7dl@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [5]:
# Set permissions for service account

gcloud_serviceusage = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{CONN_SERVICE_ACCOUNT}" --role="roles/serviceusage.serviceUsageConsumer"
"""

gcloud_bigquery = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{CONN_SERVICE_ACCOUNT}" --role="roles/bigquery.connectionUser"
"""

gcloud_aiplatform = f"""
gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member="{CONN_SERVICE_ACCOUNT}" --role="roles/aiplatform.user"
"""

print(gcloud_serviceusage)
!$gcloud_serviceusage #execute gcloud script

print(gcloud_bigquery)
!$gcloud_bigquery #execute gcloud script

print(gcloud_aiplatform)
!$gcloud_aiplatform #execute gcloud script


gcloud projects add-iam-policy-binding analytics-demo-100 --condition=None --no-user-output-enabled --member="serviceAccount:bqcx-123010175622-njc4@gcp-sa-bigquery-condel.iam.gserviceaccount.com" --role="roles/serviceusage.serviceUsageConsumer"

ERROR: (gcloud.projects.add-iam-policy-binding) User [123010175622-compute@developer.gserviceaccount.com] does not have permission to access projects instance [analytics-demo-100:setIamPolicy] (or it may not exist): Policy update access denied.

gcloud projects add-iam-policy-binding analytics-demo-100 --condition=None --no-user-output-enabled --member="serviceAccount:bqcx-123010175622-njc4@gcp-sa-bigquery-condel.iam.gserviceaccount.com" --role="roles/bigquery.connectionUser"

ERROR: (gcloud.projects.add-iam-policy-binding) User [123010175622-compute@developer.gserviceaccount.com] does not have permission to access projects instance [analytics-demo-100:setIamPolicy] (or it may not exist): Policy update access denied.

gcloud projects add-iam-p

In [13]:
!gcloud projects get-iam-policy $PROJECT_ID  \
--flatten="bindings[].members" \
--format="table(bindings.role)" \
--filter="bindings.members:$CONN_SERVICE_ACCOUNT"


ROLE
roles/aiplatform.user
roles/bigquery.connectionUser
roles/serviceusage.serviceUsageConsumer


In [14]:
#Creat a BQ dataset to store ML model and tables. This dataset must be created in the same region used by the BigQuery Cloud resource connection.

client = bigquery.Client(project=PROJECT_ID)

dataset_id = f"""{PROJECT_ID}.{DATASET_ID}"""
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION

dataset = client.create_dataset(dataset, exists_ok=True)

print(f"Dataset {dataset.dataset_id} created.")

Dataset bqml_llm created.


In [15]:
# Create a Wrapper to use BigQuery client to run query and return result


def run_bq_query(sql: str):
    """
    Input: SQL query, as a string, to execute in BigQuery
    Returns the query results or error, if any
    """
    try:
        query_job = client.query(sql)
        result = query_job.result()
        print(f"JOB ID: {query_job.job_id} STATUS: {query_job.state}")
        return result

    except Exception as e:
        raise Exception(str(e))

In [16]:
# Configure Vertex AI Model

sql = f"""
      CREATE OR REPLACE MODEL
        `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`
        REMOTE WITH CONNECTION
          `{PROJECT_ID}.{REGION}.{CONN_NAME}`
          OPTIONS ( remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1');
      """
result = run_bq_query(sql)

JOB ID: d3d0884d-fbbc-41cc-a6f3-9275f1ccba9d STATUS: DONE


In [17]:
# Using LLMs in BQML by executing the `ML.GENERATE_TEXT` function against data stored in BigQuery

PROMPT = "Describe a cat in one paragraph"

sql = f"""
          SELECT
            *
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                '{PROMPT}' AS prompt
              ),
              STRUCT
              (
                1 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output
              ));
        """
result = run_bq_query(sql)
result.to_dataframe()


JOB ID: 9701c445-d6d3-4b48-974a-5909e6ef53de STATUS: DONE


,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"A cat is a small, carnivorous mammal. It typically has a soft fur coat, a small head with large eyes, and a long tail. Cats are known for their agility and their ability to climb trees. They are also very clean animals and spend a lot of time grooming themselves. Cats are popular pets and are often kept for their companionship and their ability to catch mice.","{""blocked"":false,""categories"":[""Derogatory"",""Insult"",""Religion & Belief"",""Sexual""],""safetyRatings"":[{""category"":""Dangerous Content"",""probabilityScore"":0.1,""severity"":""NEGLIGIBLE"",""severityScore"":0.1},{""category"":""Harassment"",""probabilityScore"":0.1,""severity"":""NEGLIGIBLE"",""severityScore"":0},{""category"":""Hate Speech"",""probabilityScore"":0.1,""severity"":""NEGLIGIBLE"",""severityScore"":0},{""category"":""Sexually Explicit"",""probabilityScore"":0.2,""severity"":""NEGLIGIBLE"",""severityScore"":0.1}],""scores"":[0.1,0.1,0.5,0.2]}",,Describe a cat in one paragraph


In [19]:
# using GEMINI PRO API for creating a text generation model.

from vertexai.preview.generative_models import GenerativeModel

model = GenerativeModel("gemini-pro")

responses = model.generate_content("The opposite of hot is", stream=True)

for response in responses:
    print(response.text)
        

Cold


In [20]:
#Using Gemini API for creating a text generation model, with optional parameters.

from vertexai.preview.generative_models import GenerativeModel

model = GenerativeModel("gemini-pro")

prompt = """
You are an expert at solving word problems.

Solve the following problem:

I have three houses, each with three cats.
each cat owns 4 mittens, and a hat. Each mitten was
knit from 7m of yarn, each hat from 4m.
How much yarn was needed to make all the items?

Think about it step by step, and show your work.
"""

responses = model.generate_content(
    prompt,
    generation_config={
        "temperature": 0.1,
        "max_output_tokens": 800,
        "top_p": 1.0,
        "top_k": 40,
    },
    stream=True
    )

for response in responses:
    print(response.text)
        

1. Calculate the total number of cats:

3 houses x 3 cats
/house = 9 cats

2. Calculate the total number of mittens:


9 cats x 4 mittens/cat = 36 mittens

3. Calculate the total number of hats:

9 cats x 1 hat/
cat = 9 hats

4. Calculate the total amount of yarn needed for the mittens:

36 mittens x 7m/mitten = 2
52m

5. Calculate the total amount of yarn needed for the hats:

9 hats x 4m/hat = 36m

6. Calculate the total amount of yarn needed for all the items:

2
52m + 36m = 288m

Therefore, a total of 288 meters of yarn was needed to make all the mittens and hats for the cats in the three houses.


In [21]:
#Content creation

sql = f"""
      CREATE OR REPLACE TABLE
        `{PROJECT_ID}.{DATASET_ID}.purchases` AS
      SELECT
        u.id,
        u.first_name,
        u.email,
        u.postal_code,
        u.country,
        o.order_id,
        o.created_at,
        p.category,
        p.name
      FROM
        `bigquery-public-data.thelook_ecommerce.users` u
      JOIN (
        SELECT
          user_id,
          order_id,
          created_at,
          product_id,
          ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY created_at DESC) AS rn
        FROM
          `bigquery-public-data.thelook_ecommerce.order_items`
      ) o
      ON
        u.id = o.user_id
      JOIN
        `bigquery-public-data.thelook_ecommerce.products` p
      ON
        o.product_id = p.id
      WHERE
        o.rn = 1 AND p.category = "Active" AND u.country = "United States";
       """

result = run_bq_query(sql)

JOB ID: bec776c9-a835-4264-8b74-1edd47ccd087 STATUS: DONE


In [22]:
sql = f"""
        SELECT
            *
        FROM
            `{PROJECT_ID}.{DATASET_ID}.purchases`
        WHERE 
            first_name != 'Amy' and first_name != 'Joy'
        LIMIT
            10;
      """
result = run_bq_query(sql)
result.to_dataframe().head(10)

JOB ID: 67c43046-3e17-4d9f-b0f1-a616e5676593 STATUS: DONE


,id,first_name,email,postal_code,country,order_id,created_at,category,name
0,62057,Ana,anagallegos@example.com,53704,United States,77380,2022-06-09 06:08:11+00:00,Active,Adidas Womens Athletic Long Sleeve Hooded Tee
1,9145,Ana,analopez@example.org,7047,United States,11395,2021-08-09 05:09:22+00:00,Active,Hot Chillys Women's Micro Elite Chamois 8K Solid Tight
2,43479,Ann,annmata@example.net,80501,United States,54177,2023-10-08 13:02:50+00:00,Active,Champion Women's Absolute Bike Short
3,78531,Ann,annweber@example.net,20007,United States,97959,2023-10-28 09:17:10+00:00,Active,Women's Rayon from Bamboo Low-cut 3PK Sport Socks (5-9 Shoe)
4,92608,Joe,joeanderson@example.net,30041,United States,115506,2023-11-20 13:40:37+00:00,Active,Thorlo Unisex Anti Fatigue Boot Sock with X-STATICÂ®
5,42236,Kim,kimbrown@example.net,60637,United States,52611,2023-06-17 18:08:33+00:00,Active,Impact Fitness - Ruching Remix Jacket - Women's Stylish Workout / Running / Gym / Tennis Jacket
6,71776,Lee,leeschroeder@example.net,11434,United States,89436,2023-11-10 07:43:17+00:00,Active,Diamond Supply Co Mod Crew Sweatshirt - Men's
7,95921,Pam,pamandrews@example.net,89002,United States,119593,2023-07-15 11:45:11+00:00,Active,Port Authority Ladies Puffy Vest. L709
8,61748,Adam,adamhart@example.com,75248,United States,76988,2023-10-06 12:24:04+00:00,Active,prAna Men's Stretch 30-Inch Inseam Zion Pant
9,19246,Adam,adamjones@example.net,75068,United States,23880,2022-11-05 15:25:16+00:00,Active,PUMA Men's Ferrari Logo Tee


In [23]:
def run_bq_query_to_table(
    destination_table: str,
    query: str,
    project_id: str,
    location: str,
):
    """
    A helper function to create n daily feature tables over a date interval, 
    where n stands for the number of days in the backward interval.
    Then return names of feature tables in a list.
    Args:
        features_table_name: name of the feature table
        query: query to create the feature table
        start_date: the date to start creating features
        backward_ndays: number of days going backward from start date
        project_id: project id
        region: region
    Returns:
        features_table_names: list of feature tables
    """

    try:
        # Create a BigQuery client.
        bq_client = bigquery.Client(project=project_id, location=location)

        # Create the query job.
        job_config = bigquery.QueryJobConfig(
            destination=f"{project_id}.{destination_table}", write_disposition="WRITE_TRUNCATE"
        )

        # Run the query to create daily feature table.
        job = bq_client.query(query, job_config=job_config)
        df = job.result().to_arrow().to_pandas()
        
        print(f"Created BQ table: {destination_table}")

    except RuntimeError as error:
        print(error)

    return df

In [24]:
PROMPT_PART1 = "A user bought a product with this description: "
PROMPT_PART2 = ' Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example "We hope you are enjoying your new t-shirt". '
PROMPT_PART3 = "Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. "
PROMPT_PART4 = "Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code "
PROMPT_PART5 = '. Do not mention the brand of the product, just sign off the email with "-TheLook." Address the email to: '

sql = f"""
          SELECT
            prompt,
            ml_generate_text_llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT_PART1}',name,'{PROMPT_PART2}','{PROMPT_PART3}','{PROMPT_PART4}',postal_code,'{PROMPT_PART5}',first_name) AS prompt
              FROM
                `{PROJECT_ID}.{DATASET_ID}.purchases`
              WHERE 
                first_name != 'Amy' and first_name != 'Joy'
              LIMIT
                5),
              STRUCT(0.8 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)

run_bq_query_to_table(
    destination_table=f"{DATASET_ID}.genAIResponse",
    query=sql,
    project_id=PROJECT_ID,
    location=REGION,
)
result.to_dataframe()

JOB ID: 10292fbd-de35-42ce-bf14-30727d3eab65 STATUS: DONE
Created BQ table: bqml_llm.genAIResponse


,prompt,ml_generate_text_llm_result
0,"A user bought a product with this description: Adidas Womens Athletic Long Sleeve Hooded Tee Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example ""We hope you are enjoying your new t-shirt"". Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code 53704. Do not mention the brand of the product, just sign off the email with ""-TheLook."" Address the email to: Ana","Subject: Your New Purchase\n\nHi Ana,\n\nWe hope you're enjoying your new activewear!\n\nHere are two fun local outdoor activities you can pursue with your new purchase:\n\n- Take a hike at nearby Lapham Peak State Park\n- Go for a bike ride on the Oak Leaf Trail\n\nTo thank you for shopping with us, we'd like to offer you a 10% discount on your next purchase. Simply use the coupon code RETURN10 at checkout.\n\nWe appreciate your business and look forward to serving you again soon.\n\n-TheLook"
1,"A user bought a product with this description: Hot Chillys Women's Micro Elite Chamois 8K Solid Tight Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example ""We hope you are enjoying your new t-shirt"". Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code 7047. Do not mention the brand of the product, just sign off the email with ""-TheLook."" Address the email to: Ana","Subject: Your New Athletic Wear\n\nDear Ana,\n\nWe hope you're enjoying your new athletic wear! We're excited to have you as a part of our community of outdoor enthusiasts.\n\nAs a token of our appreciation, we'd like to offer you a 10% discount on your next purchase with us. Simply use the code RETURN10 at checkout.\n\nWe're also happy to provide you with two local outdoor activities that you can pursue with your new purchase:\n\n- Hiking in Fontainebleau State Park: This park is located just a short drive from your home and offers miles of hiking trails for all levels of experience.\n- Kayaking in Lake Pontchartrain: This lake is a great place to explore by kayak, and there are several rental shops located nearby.\n\nWe hope you enjoy your new athletic wear and we look forward to seeing you in our store again soon.\n\n-TheLook"
2,"A user bought a product with this description: Champion Women's Absolute Bike Short Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example ""We hope you are enjoying your new t-shirt"". Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code 80501. Do not mention the brand of the product, just sign off the email with ""-TheLook."" Address the email to: Ann","Subject: Your New Champion Purchase\n\nHi Ann,\n\nWe hope you're enjoying your new activewear!\n\nHere are two fun local outdoor activities you can pursue with your new purchase: \n\n- Take a bike ride on the Boulder Creek Path\n- Go for a hike in Chautauqua Park\n\nWe'd love to see you back in our store. Use the coupon code RETURN10 for 10% off your next purchase.\n\n-TheLook"
3,"A user bought a product with this description: Women's Rayon from Bamboo Low-cut 3PK Sport Socks (5-9 Shoe) Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example ""We hope you are enjoying your new t-shirt"". Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. Provide two local outdoor ac

In [19]:
PROMPT_PART1 = "A user bought a product with this description: "
PROMPT_PART2 = ' Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example "We hope you are enjoying your new t-shirt". '
PROMPT_PART3 = "Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. "
PROMPT_PART4 = "Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code "
PROMPT_PART5 = '. Do not mention the brand of the product, just sign off the email with "-TheLook." Address the email to: '

sql = f"""
          SELECT
            prompt,
            ml_generate_text_llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT_PART1}',name,'{PROMPT_PART2}','{PROMPT_PART3}','{PROMPT_PART4}',postal_code,'{PROMPT_PART5}',first_name) AS prompt
              FROM
                `{PROJECT_ID}.{DATASET_ID}.purchases`
              WHERE 
                first_name != 'Amy' and first_name != 'Joy'
              LIMIT
                5),
              STRUCT(0.8 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)

result.to_dataframe()

JOB ID: 7a73f500-156c-4b69-ab3d-a2eae4e174f9 STATUS: DONE


,prompt,ml_generate_text_llm_result
0,"A user bought a product with this description: adidas Men's Climalite II No Show Sock (2-Pack)(Shoe Size 6-12) Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example ""We hope you are enjoying your new t-shirt"". Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code 77088. Do not mention the brand of the product, just sign off the email with ""-TheLook."" Address the email to: Jay","Subject: We hope you're enjoying your new socks!\n\nHi Jay,\n\nWe hope you're enjoying your new socks! \n\nAs a token of our appreciation, we'd like to offer you a 10% discount on your next purchase with the code RETURN10.\n\nWe have a wide selection of outdoor gear and apparel to choose from, so you can find the perfect items for your next adventure.\n\nHere are two local outdoor activities that you could pursue with your new socks:\n\n1. Visit the Houston Arboretum and Nature Center. This 155-acre park is located in the heart of Houston and features a variety of trails, gardens, and exhibits.\n\n2. Go for a hike at the Terry Hershey Park. This 500-acre park is located in Katy and offers a variety of trails for all levels of hikers.\n\nWe hope to see you again soon!\n\n-TheLook"
1,"A user bought a product with this description: Columbia Women's Glacial Fleece Iii 1/2 Zip Jacket Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example ""We hope you are enjoying your new t-shirt"". Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code 28208. Do not mention the brand of the product, just sign off the email with ""-TheLook."" Address the email to: Kim","Subject: We hope you're enjoying your new fleece!\n\nHi Kim,\n\nWe hope you're enjoying your new fleece! We know you'll love the warmth and comfort it provides, whether you're out on the trails or just running errands around town.\n\nAs a token of our appreciation, we'd like to offer you a 10% discount on your next purchase with the code RETURN10. We have a wide selection of outdoor gear and apparel to choose from, so you're sure to find something you love.\n\nHere are two local outdoor activities you could pursue with your new fleece:\n\n1. Visit Crowders Mountain State Park, located just 30 minutes from Charlotte. The park offers over 25 miles of hiking trails, as well as camping, fishing, and swimming.\n\n\n2. Go kayaking on the Catawba River. There are several outfitters in the area that rent kayaks and offer guided tours.\n\nWe hope to see you again soon!\n\n-TheLook"
2,"A user bought a product with this description: Life is Good Women's Jackie Lounge Mug Long Sleeve Crusher Tee Write a follow up marketing email mentioning the high-level product category of their purchase in one word, for example ""We hope you are enjoying your new t-shirt"". Encourage the individual to shop with the store again using the coupon code RETURN10 for 10% off their next purchase. Provide two local outdoor activities they could pursue with their new purchase. They live in the zip code 85711. Do not mention the brand of the product, just sign off the email with ""-TheLook."" Address the email to: Kim","Subject: We hope you're enjoying your new tee!\n\nHi Kim,\n\nWe hope you're enjoying your new tee! \n\nWe'd like to thank you for shopping with us and would love to offer you a 10% discount on your next purchase with the code RETURN10.\n\nSince you live in the 85711 zip code, here are two local outdoor activities you could pursue with your new tee: \n\n- Visit the Scottsdale Civic Center Mall for some shopping and dining.\n- 

In [64]:
PROJECT_ID2 = "analytics-demo-100"
DATASET_ID2 = "SF_Talent"

sql = f"""
    SELECT *
    FROM `{PROJECT_ID2}.{DATASET_ID2}.Talent_bq`
    WHERE
        User_ID_Address IS NOT NULL
        AND User_ID_Language IS NOT NULL
        AND User_ID_LeadExperience IS NOT NULL
        AND User_ID_Mobility IS NOT NULL
        AND User_ID_Phone IS NOT NULL
        AND User_ID_PreviousWork IS NOT NULL
        AND User_ID != 'akuwa' and User_ID != 'henomoto' and User_ID != 'tkawasaki' and User_ID != 'jsakata' and User_ID != '107015'and User_ID != '107010'
        
    LIMIT 1000;
"""
run_bq_query_to_table(
    destination_table=f"{DATASET_ID2}.Talentdata",
    query=sql,
    project_id=PROJECT_ID2,
    location=REGION,
)


result = run_bq_query(sql)
result.to_dataframe()


Created BQ table: SF_Talent.Talentdata
JOB ID: 33b7e676-3169-42e4-9f8d-9982ab403c36 STATUS: DONE


,User_ID,Pay_Grade,Job_Title,Employee_Name,Business_Impact,Retention_Risk,Manager_Name,Position_Entry_Date,Position_Entry_Year,User_ID_Phone,...,User_ID_Mobility,Location_Desired,EmployeeCountry,Willingness,User_ID_LeadExperience,Comments,Experience,User_ID_Language,Language,Level
0,107009,GR-11,Production Director,Yi Lu,High,High,Molly Tang,2009-01-12,2009,107009,...,107009,日本,China,Yes,107009,#,Leadership of cross-functional task forces or projects,107009,English (UK),Fluent
1,107004,GR-12,VP Operations,Molly Tang,High,High,Xiao-niing Sun,1996-01-01,1996,107004,...,107004,Australia,China,Yes,107004,#,Business turnaround,107004,Spanish,Intermediate
2,104010,GR-08,Program Management Office,Silvia Candido,High,High,Paulo Moreira,1997-01-01,1997,104010,...,104010,"Tokyo, Japan",Brazil,Yes,104010,#,Leadership of cross-functional task forces or projects,104010,Japanese,Intermediate
3,104009,GR-10,Engineering Manager,Eduardo Oliveria,High,Medium,Paulo Moreira,2009-12-01,2009,104009,...,104009,"Tokyo, Japan",Brazil,Yes,104009,#,Leadership of cross-functional task forces or projects,104009,Italian,Intermediate
4,104004,GR-12,VP Operations,Marlise Goncalves,High,High,Andre Catarino,1996-01-01,1996,104004,...,104004,London,Brazil,Yes,104004,#,Operational focus and accountability,104004,English (UK),Fluent
5,104036,GR-11,Production Director,Paulo Moreira,High,High,Marlise Goncalves,2009-05-01,2009,104036,...,104036,Japan,Brazil,Yes,104036,#,Business turnaround,104036,English (UK),Fluent
6,102010,GR-08,Program Management Office,Karen Bernau,Low,Low,Hubert Maigne,1997-01-01,1997,102010,...,102010,"Tokyo, Japan",France,Yes,102010,#,Leadership of cross-functional task forces or projects,102010,Japanese,Intermediate
7,102036,GR-10,Engineering Manager,Gilbert Grape,High,Medium,Hubert Maigne,2009-12-01,2009,102036,...,102036,"Beijing, China",France,Yes,102036,#,Leadership of cross-functional task forces or projects,102036,Chinese,Intermediate
8,102004,GR-12,VP Operations,Amelie Verrier,High,High,Jacques Jouan,1996-01-01,1996,102004,...,102004,Madrid,France,Yes,102004,#,Operational focus and accountability,102004,Spanish,Intermediate
9,102009,GR-11,Production Director,Hubert Maigne,High,High,Amelie Verrier,2009-01-12,2009,102009,...,102009,Japan,France,Yes,102009,#,Leadership of cross-functional task forces or projects,102009,English (UK),Fluent


In [67]:
PROMPT_PART1 = " Generate a summary of the employee profile, for example Meet Sarah Smith, a dedicated employee of our organization. She currently serves as a Senior Project Manager. The employee name and position:  "
PROMPT_PART2 = " Create a narrative that includes the employee date of birth, state of residence, and country of residence, for example Sarah was born on January 15, 1985, and resides in the bustling city of Seattle, Washington. You can reach her at sarah.smith@email.com or +1 (555) 123-4567. Sarah is has an intermediate English proficiency. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:"
PROMPT_PART3 = " Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: "


sql = f"""
          SELECT
            User_ID,
            prompt,
            ml_generate_text_llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
              User_ID,
                CONCAT('{PROMPT_PART1}', Employee_Name, Job_Title, '{PROMPT_PART2}', BirthDATE, State, Country,Email,Phone_number,Language,Level,'{PROMPT_PART3}',Pay_Grade, Manager_Name) AS prompt
              FROM
                `{PROJECT_ID2}.{DATASET_ID2}.Talentdata`
              ),
              STRUCT(1 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)

run_bq_query_to_table(
    destination_table=f"{DATASET_ID2}.TalentdataResponse",
    query=sql,
    project_id=PROJECT_ID2,
    location=REGION,
)

result.to_dataframe()

JOB ID: ea1a982e-fb10-4214-a451-14a49db34e2c STATUS: DONE
Created BQ table: SF_Talent.TalentdataResponse


,User_ID,prompt,ml_generate_text_llm_result
0,106005,"Generate a summary of the employee profile, for example Meet Sarah Smith, a dedicated employee of our organization. She currently serves as a Senior Project Manager. The employee name and position: Anastasia MillarSr. Director Operations Create a narrative that includes the employee date of birth, state of residence, and country of residence, for example Sarah was born on January 15, 1985, and resides in the bustling city of Seattle, Washington. You can reach her at sarah.smith@email.com or +1 (555) 123-4567. Sarah is has an intermediate English proficiency. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1980-11-30New South WalesAustraliaAnastasia.Millar@bestrunsap.com61-02-7010 8705SpanishIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-12Richard Redman","Anastasia Millar is a dedicated employee of our organization, currently serving as a Sr. Director Operations. She was born on November 30, 1980, and resides in New South Wales, Australia. You can reach her at Anastasia.Millar@bestrunsap.com or 61-02-7010 8705. Anastasia is an intermediate English speaker.\nAnastasia is a GR-12 employee and reports to Richard Redman."
1,100115,"Generate a summary of the employee profile, for example Meet Sarah Smith, a dedicated employee of our organization. She currently serves as a Senior Project Manager. The employee name and position: Mya CooperVP Operations Create a narrative that includes the employee date of birth, state of residence, and country of residence, for example Sarah was born on January 15, 1985, and resides in the bustling city of Seattle, Washington. You can reach her at sarah.smith@email.com or +1 (555) 123-4567. Sarah is has an intermediate English proficiency. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1980-11-30PennsylvaniaUnited Stateshyagci@deloitte.de1-732-555-7222SpanishIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-14John Williams","Mya Cooper is a dedicated employee who currently serves as the VP Operations. She was born on November 30, 1980, and resides in Pennsylvania, United States. Mya can be reached at hyagci@deloitte.de or +1 (732) 555-7222. She has an intermediate proficiency in Spanish. Mya is a GR-14 employee and reports to John Williams."
2,101005,"Generate a summary of the employee profile, for example Meet Sarah Smith, a dedicated employee of our organization. She currently serves as a Senior Project Manager. The employee name and position: Sandra GrayVP Operations Create a narrative that includes the employee date of birth, state of residence, and country of residence, for example Sarah was born on January 15, 1985, and resides in the bustling city of Seattle, Washington. You can reach her at sarah.smith@email.com or +1 (555) 123-4567. Sarah is has an intermediate English proficiency. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1980-11-30#United KingdomSandra.Gray@bestrunsap.com44-3069-990060SpanishIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-12Graham Goddard","Sandra Gray is a dedicated employee who currently serves as the VP Operations. She was born on November 30, 1980, and resides in the United Kingdom. Sandra can be reached at Sandra.Gray@bestrunsap.com or 44-3069-990060. She has an intermediate proficiency in Spanish. Sandra is in pay grade GR-12 and reports to Graham Goddard."
3,SwamiN,"Generate a summary of the employee profile, for example Meet Sarah Smith, a dedicated employee of our organization. She currently serves as a Senior Project Manager. The employee name and position: Narayan 

In [35]:
PROMPT_PART1 = " Generate a summary of the employee profile. The employee name and position:  "
PROMPT_PART2 = " Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:"
PROMPT_PART3 = " Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: "


sql = f"""
          SELECT
            User_ID,
            prompt,
            ml_generate_text_llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                CONCAT('{PROMPT_PART1}', Employee_Name, Job_Title, '{PROMPT_PART2}', BirthDATE, State, Country,Email,Phone_number,Language,Level,'{PROMPT_PART3}',Pay_Grade, Manager_Name) AS prompt
              FROM
                `{PROJECT_ID2}.{DATASET_ID2}.Talentdata`
              LIMIT
                5),
              STRUCT(1 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output));
        """
result = run_bq_query(sql)

result.to_dataframe()

JOB ID: 5de1defc-d781-4913-a240-0da778ed1917 STATUS: DONE


,prompt,ml_generate_text_llm_result
0,"Generate a summary of the employee profile. The employee name and position: 敦 桑原東日本営業部長 Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1981-05-21TokyoJapanatsushi.kuwahara@bestrunsap.com81-080-1234-1234GermanBeginner Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-12誠 田中","敦 桑原 is an employee at BestRunSAP. 敦桑原 was born on 1981-05-21 in Tokyo, Japan. 敦桑原 currently resides in Tokyo, Japan. 敦桑原's email address is atsushi.kuwahara@bestrunsap.com, and their phone number is 81-080-1234-1234. 敦桑原 speaks German and is a beginner in the language. 敦桑原 is a GR-12 employee and reports to 誠 田中."
1,"Generate a summary of the employee profile. The employee name and position: 健史 河崎北海道支社長 Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1977-05-01TokyoJapantakeshi.kawasaki@bestrunsap.com81-3-1294-1431English (US)Intermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-08敦 桑原","健史 河崎北海道支社長 was born on 1977-05-01 in Tokyo, Japan. He currently resides in Japan and his email address is takeshi.kawasaki@bestrunsap.com. His phone number is 81-3-1294-1431 and he speaks English (US) at an intermediate level. 健史 河崎北海道支社長 is a GR-08 employee and his manager is 敦 桑原."
2,"Generate a summary of the employee profile. The employee name and position: 宏典 榎本北関東支社長 Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1981-03-05#JapanHironori.Enomoto@bestrun.com81-3-1294-1432English (US)Intermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-08敦 桑原","宏典 榎本 is a 支社長 at ベストラン. 宏典 was born on 1981-03-05 in Japan. 宏典 currently lives in Japan. 宏典's email address is Hironori.Enomoto@bestrun.com, and their phone number is 81-3-1294-1432. 宏典 speaks English (US) at an intermediate level. 宏典 is a GR-08 and reports to 敦 桑原."
3,"Generate a summary of the employee profile. The employee name and position: 純子 坂田南関東支社長 Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1965-01-05TokyoJapanJunko.Sakata@bestrun.com81-3-1345-1114PortugueseIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-08敦 桑原","純子 坂田 is the South Kanto Branch Manager. She was born on January 5th, 1965 in Tokyo, Japan and currently resides there. Her email address is Junko.Sakata@bestrun.com, her phone number is 81-3-1345-1114, and she speaks Portuguese at an intermediate level. \n\n純子 坂田 is a GR-08 employee and reports to 敦 桑原."
4,"Generate a summary of the employee profile. The employee name and position: Eduardo OliveriaEngineering Manager Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1981-01-19São PauloBrazilEduardo.Oliveria@bestrunsap.com55-11-5503-2532ItalianIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-10Paulo Moreira","Eduardo Oliveria is an Engineering Manager at BestRunSAP. He was born on January 19th, 1981 in São Paulo, Brazil and currently res

In [65]:
PROMPT_PART1 = " Generate a summary of the employee profile. The employee name and position:  "
PROMPT_PART2 = " Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:"
PROMPT_PART3 = " Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: "


sql = f"""
          SELECT
            User_ID,
            prompt,
            ml_generate_text_llm_result
          FROM
            ML.GENERATE_TEXT(
              MODEL `{PROJECT_ID}.{DATASET_ID}.{LLM_MODEL_NAME}`,
              (
              SELECT
                User_ID,
                CONCAT('{PROMPT_PART1}', Employee_Name, Job_Title, '{PROMPT_PART2}', BirthDATE, State, Country,Email,Phone_number,Language,Level,'{PROMPT_PART3}',Pay_Grade, Manager_Name) AS prompt
              FROM
                `{PROJECT_ID2}.{DATASET_ID2}.Talentdata`
              )
            ,
              STRUCT(1 AS temperature,
                1024 AS max_output_tokens,
                0.8 AS top_p,
                40 AS top_k,
                TRUE AS flatten_json_output))
             ;
        """
result = run_bq_query(sql)

result.to_dataframe()

JOB ID: 60cf66fe-e3a0-494d-89e1-3afd6f223302 STATUS: DONE


,User_ID,prompt,ml_generate_text_llm_result
0,106005,"Generate a summary of the employee profile. The employee name and position: Anastasia MillarSr. Director Operations Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1980-11-30New South WalesAustraliaAnastasia.Millar@bestrunsap.com61-02-7010 8705SpanishIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-12Richard Redman","Anastasia Millar is a Sr. Director Operations at BestRunSAP. She was born on November 30th, 1980 in New South Wales, Australia. She currently resides in Australia. Her email address is Anastasia.Millar@bestrunsap.com, her phone number is 61-02-7010 8705, and she speaks Spanish at an intermediate level. Her job grade is GR-12 and her manager is Richard Redman."
1,100115,"Generate a summary of the employee profile. The employee name and position: Mya CooperVP Operations Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1980-11-30PennsylvaniaUnited Stateshyagci@deloitte.de1-732-555-7222SpanishIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-14John Williams","Mya Cooper is the VP of Operations. She was born on November 30th, 1980. She lives in Pennsylvania, United States. Her email is hyagci@deloitte.de, her phone number is 1-732-555-7222, and she speaks Spanish. She is an intermediate level employee with a job grade of GR-14. Her manager is John Williams."
2,101005,"Generate a summary of the employee profile. The employee name and position: Sandra GrayVP Operations Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1980-11-30#United KingdomSandra.Gray@bestrunsap.com44-3069-990060SpanishIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-12Graham Goddard","Sandra Gray is the VP of Operations at BestRunSAP. She was born on November 30th, 1980, and resides in the United Kingdom. Her email address is Sandra.Gray@bestrunsap.com, her phone number is 44-3069-990060, and she speaks Spanish at an intermediate level. Sandra is a GR-12 employee and reports to Graham Goddard."
3,SwamiN,"Generate a summary of the employee profile. The employee name and position: Narayan SwamiCapacity Planning Manager Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1986-06-03#Singaporenarayan.swami@bestrunsap.com+65-0-65762425SpanishIntermediate Write a narrative that includes the employee job grade and the name of their manager. The employee pay grade and manager: GR-08Thomas Lin",Employee Profile\n\nName: Narayan Swami\n\nPosition: Capacity Planning Manager\n\nDate of Birth: 1986-06-03\n\nState of Residence: Singapore\n\nCountry of Residence: Singapore\n\nEmail: narayan.swami@bestrunsap.com\n\nPhone Number: +65-0-65762425\n\nLanguage: Spanish\n\nLevel: Intermediate\n\nJob Grade: GR-08\n\nManager: Thomas Lin
4,102004,"Generate a summary of the employee profile. The employee name and position: Amelie VerrierVP Operations Create a narrative that includes the employee date of birth, state of residence, and country of residence. The employee date of birth, state of residence, country of residence, Email, Phone number, Language, Level:1980-11-30#FranceAmelie.Verrier@bestrunsap.com33-02-81 54 7